In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


from tkinter import *
import cv2
import re
from tkinter import filedialog
from rembg import remove
from PIL import Image
from easyocr import easyocr
import numpy as np
import pytesseract
import string
# from OCR_Endpoint import settings

data = {"first name": "0",
            "seconed name": "0",
            "address": "0",
            "id": "0",
            "error": 0}

    #print(pytesseract.tesseract_cmd)
    #pytesseract.tesseract_cmd = settings.BASE_DIR+'Tesseract-OCR/tesseract.exe'
try:
    wi=Tk()
    wi.title("choose image")
    file=filedialog.askopenfilename()
    path=open(file, 'r')
    name=path.name
    print(name)
    wi.mainloop()
    path.close()
    input_path = name
    output_path = 'moraaa.png'
    #sharp_path = 'sharp.png'
    input = Image.open(input_path)
    output = remove(input)
    output.save(output_path)

    img = cv2.imread(output_path)
    #resized=cv2.resize(img,(350,450))
    blurred = cv2.blur(img, (5,5))
    kernel = np.array([[-1,-1,-1] ,[-1,9,-1],[-1,-1,-1]])
    sharpened = cv2.filter2D(blurred, -1, kernel)
    canny = cv2.Canny(sharpened, 50, 200)
    pts = np.argwhere(canny>0)
    y1,x1 = pts.min(axis=0)
    y2,x2 = pts.max(axis=0)
    cropped = img[y1:y2, x1:x2]
    w,h,c=cropped.shape
    o=int(w/2)
    i=int(h/2.5)
    n=int(h/6)
    cr=cropped[n-13:i+7,o:]
    cropped_img=cropped[i+10:,o+10:]
    cv2.imwrite("newimg.png",cropped_img)
    text=pytesseract.image_to_string(cr,lang='ara',config='--psm 11 --oem 3')
    splited=text.split('\n')
    arabic_digits = ["٠", "١", "٢", "٣", "٤", "٥", "٦", "٧", "٨", "٩"]
    pun=set(string.punctuation)

    state=0
    if len(text.split('\n'))==8:
        state=1
        print(state)
        firstname=splited[0]
        secondname=splited[2]
        address=splited[4]+" "+splited[6]
        data["first name"] = firstname
        data["seconed name"] = secondname
        data["address"] = address
        for i in data:
            if i == None:
                data["error"] = "1"
                break
            else:
                imgs = cv2.imread('newimg.png',0)
                gauss = cv2.GaussianBlur(imgs, (7,7), 0)
                unsharp_image = cv2.addWeighted(imgs, 2, gauss, -1, 0)
                s=easyocr.Reader(['ar','ar'])
                o=s.readtext(unsharp_image, detail = 0,text_threshold = 0.27
                ,width_ths = 0.8,low_text= .008)
                if len(o) == 1:
                    data["id"] = o[0]
                elif len(o) == 0:
                    data["id"] = "0"
                elif len(o) > 1:
                    data["id"]=max(o, key=len)
            break

    elif state==0:
        state=2
        imgs = cv2.imread('newimg.png',0)
        gauss = cv2.GaussianBlur(imgs, (7,7), 0)
        unsharp_image = cv2.addWeighted(imgs, 2, gauss, -1, 0)
        s=easyocr.Reader(['ar','ar'])
        d=s.readtext(cr, detail = 0,text_threshold = 0.18
        ,width_ths = 0.9,low_text= 0.17)
        for i in d:
            if i in arabic_digits:
                break
            else:
                my_data = ','.join(d)
                split_list = my_data.split(',')
                data["first name"] = split_list[0]
                data["seconed name"] = split_list[1]
                data["address"] = str(d[2:])
                for i in data["address"]:
                    if i == "[" or i == "]" or i == " ' ":
                        data["address"] = data["address"].replace("[", "").replace("]", "").replace(",","").replace("'","")
        o=s.readtext(unsharp_image, detail = 0,text_threshold = 0.27
        ,width_ths = 0.8,low_text= 0.008)
        print(state)
        if o == None or d == None:
            state = 4
            data["error"] = "1"
        else:
            if len(o) == 1:
                data["id"] = o[0]
            elif len(o) == 0:
                data["id"] = "0"
            elif len(o) > 1:
                    data["id"]=max(o, key=len)

    elif state == 4:
        data["error"] = "1"
    if len(data["id"])<20:
        data["error"]="1"
    for i in data["first name"]:
        if i in arabic_digits or i in pun:
            data["first name"] = data["first name"].replace(i, "")
    for i in data["seconed name"]:
        if i in arabic_digits or i in pun:
            data["seconed name"] = data["seconed name"].replace(i, "")

    ar=['ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ي']
    for i in data["id"]:
        if i in ar or i in pun:
            data["id"]=data["id"].replace(i, "")
  

    arabic_string = data["id"]

    # Use regular expression to extract individual numbers
    regex = r'[٠-٩]+'
    matches = re.findall(regex, arabic_string)

    # Reverse the order of the matches
    matches.reverse()

    # Concatenate the matches into a single string
    concatenated_string = ''.join(matches)

    # Convert the concatenated string to an integer
    integer_value = int(concatenated_string)
    data["id"]=integer_value
    

except:
    data['id']=0
    data['error']=1

for key, value in data.items():
    print(key, value)
